## تولید مکالمات مختلف برای هر بیماری



In [ ]:
import json
import random
from typing import List, Dict

class MedicalConversationGenerator:
    def __init__(self):
        self.conversation_types = [
            "patient_initial_complaint",     
            "symptom_inquiry",               
            "diagnosis_explanation",          
            "treatment_discussion",         
            "follow_up_questions",           
            "prevention_advice",              
            "emergency_situations"            
        ]
        
        self.user_personas = [
            "بیمار جوان نگران",
            "والدین کودک بیمار", 
            "سالمند با سوالات متعدد",
            "بیمار با سطح تحصیلات بالا",
            "بیمار با دانش محدود پزشکی"
        ]

    def generate_conversation_prompt(self, disease_info: Dict, conversation_type: str, persona: str) -> str:
        base_prompt = f"""
        بر اساس اطلاعات بیماری زیر:
        {json.dumps(disease_info, ensure_ascii=False, indent=2)}
        
        یک مکالمه طبیعی و واقعی بین پزشک و {persona} تولید کن.
        نوع مکالمه: {conversation_type}
        
        قوانین مهم:
        1. مکالمه باید کاملاً طبیعی باشد
        2. پزشک باید حرفه‌ای و دلسوزانه پاسخ دهد
        3. اطلاعات پزشکی دقیق باشد
        4. زبان ساده و قابل فهم استفاده شود
        5. سوالات متنوع و واقعی باشد
        
        فرمت خروجی:
        بیمار: [سوال/شکایت]
        پزشک: [پاسخ تخصصی]
        بیمار: [سوال پیگیری]
        پزشک: [پاسخ]
        ...
        """
        return base_prompt

    def generate_diverse_conversations(self, disease_data: List[Dict], conversations_per_disease: int = 10):
        all_conversations = []
        
        for disease in disease_data:
            for i in range(conversations_per_disease):
                conv_type = random.choice(self.conversation_types)
                persona = random.choice(self.user_personas)
                
                prompt = self.generate_conversation_prompt(disease, conv_type, persona)
                
                conversation = self.call_llm_api(prompt)
                
                all_conversations.append({
                    "disease": disease["disease_name"],
                    "conversation_type": conv_type,
                    "persona": persona,
                    "conversation": conversation,
                    "metadata": disease
                })
                
        return all_conversations

## تولید انواع مختلف سوالات

In [ ]:
class QuestionVariationGenerator:
    def __init__(self):
        self.question_patterns = {
            "symptoms": [
                "علائم {disease} چیست؟",
                "چطور بفهمم که {disease} دارم؟", 
                "نشانه‌های {disease} کدامند؟",
                "اولین علائم {disease} چه زمانی ظاهر می‌شود؟"
            ],
            "causes": [
                "علت {disease} چیست؟",
                "چرا {disease} پیدا می‌شود؟",
                "چه چیزی باعث {disease} می‌شود؟"
            ],
            "treatment": [
                "درمان {disease} چیست؟",
                "چطور {disease} را درمان کنم؟",
                "بهترین روش درمان {disease} کدام است؟"
            ]
        }

    def generate_question_variations(self, disease_name: str, count: int = 50):
        variations = []
        
        for category, patterns in self.question_patterns.items():
            for pattern in patterns:
                base_question = pattern.format(disease=disease_name)
                
                variation_prompt = f"""
                این سوال پایه را به {count//len(patterns)} روش مختلف و طبیعی بازنویسی کن:
                "{base_question}"
                
                تنوع در:
                - انتخاب کلمات
                - ساختار جمله  
                - سطح رسمی/غیررسمی
                - طول جمله
                
                هر بازنویسی در خط جداگانه باشد.
                """
                
                variations_text = self.call_llm_api(variation_prompt)
                variations.extend(variations_text.split('\n'))
                
        return variations

## تولید سناریوهای پیچیده

In [ ]:
class ComplexScenarioGenerator:
    def generate_multi_symptom_scenarios(self, diseases_data):

        
        prompt = f"""
        بر اساس این لیست بیماری‌ها:
        {json.dumps(diseases_data, ensure_ascii=False)}
        
        سناریوهای پیچیده‌ای تولید کن که شامل:
        1. بیماری با علائم مشترک
        2. عوارض چندگانه
        3. بیماری‌های همزمان
        4. تشخیص افتراقی
        
        هر سناریو شامل مکالمه کامل بین پزشک و بیمار باشد.
        """
        
        return self.call_llm_api(prompt)

    def generate_age_gender_variations(self, base_conversation, demographics):
        
        variations = []
        for demo in demographics:
            variation_prompt = f"""
            این مکالمه پزشکی را برای {demo['age']} ساله {demo['gender']} تطبیق ده:
            
            {base_conversation}
            
            در نظر بگیر:
            - تفاوت‌های سنی در علائم
            - مسائل خاص جنسیتی
            - نحوه صحبت متناسب با سن
            """
            
            varied_conversation = self.call_llm_api(variation_prompt)
            variations.append(varied_conversation)
            
        return variations

## کنترل کیفیت و فیلتر کردن

In [ ]:
class DataQualityController:
    def validate_conversation(self, conversation: str) -> bool:        
        validation_prompt = f"""
        این مکالمه پزشکی را از نظر موارد زیر بررسی کن:
        
        {conversation}
        
        معیارهای ارزیابی:
        1. دقت اطلاعات پزشکی (0-10)
        2. طبیعی بودن مکالمه (0-10)  
        3. سطح مناسب زبان (0-10)
        4. کامل بودن پاسخ‌ها (0-10)
        
        خروجی فقط یک عدد بین 0 تا 10 باشد (میانگین نمرات).
        """
        
        score = float(self.call_llm_api(validation_prompt))
        return score >= 7.0  
    def remove_duplicates(self, conversations: List[Dict]) -> List[Dict]:
        """حذف مکالمات تکراری"""
        unique_conversations = []
        seen_content = set()
        
        for conv in conversations:
            content_hash = hash(conv['conversation'][:200])
            
            if content_hash not in seen_content:
                seen_content.add(content_hash)
                unique_conversations.append(conv)
                
        return unique_conversations

##  اسکریپت کامل تولید داده

In [ ]:
class MedicalDatasetGenerator:
    def __init__(self):
        self.conversation_gen = MedicalConversationGenerator()
        self.question_gen = QuestionVariationGenerator()
        self.scenario_gen = ComplexScenarioGenerator()
        self.quality_controller = DataQualityController()
        
    def generate_complete_dataset(self, extracted_diseases: List[Dict], 
                                target_size: int = 10000):
        
        print("شروع تولید دیتاست...")
        
        conversations_per_disease = target_size // len(extracted_diseases)
        
        all_data = []
        
        print("تولید مکالمات پایه...")
        base_conversations = self.conversation_gen.generate_diverse_conversations(
            extracted_diseases, conversations_per_disease
        )
        
        print("تولید سناریوهای پیچیده...")
        complex_scenarios = self.scenario_gen.generate_multi_symptom_scenarios(
            extracted_diseases
        )
        
        all_data.extend(base_conversations)
        
        print("کنترل کیفیت...")
        filtered_data = []
        for conv in all_data:
            if self.quality_controller.validate_conversation(conv['conversation']):
                filtered_data.append(conv)
        
        print("حذف موارد تکراری...")
        final_data = self.quality_controller.remove_duplicates(filtered_data)
        
        print(f"دیتاست نهایی: {len(final_data)} مکالمه")
        
        return final_data

    def save_dataset(self, dataset, filename):
        
        with open(f"{filename}.json", 'w', encoding='utf-8') as f:
            json.dump(dataset, f, ensure_ascii=False, indent=2)
        
        df = pd.DataFrame(dataset)
        df.to_csv(f"{filename}.csv", index=False, encoding='utf-8-sig')
        
        print(f"دیتاست ذخیره شد: {filename}")

generator = MedicalDatasetGenerator()

with open('extracted_medical_data.json', 'r', encoding='utf-8') as f:
    diseases_data = json.load(f)

dataset = generator.generate_complete_dataset(diseases_data, target_size=5000)

generator.save_dataset(dataset, 'medical_conversations_dataset')